<a href="https://colab.research.google.com/github/taniyasinghsisodia/Deep-Learning-Projects/blob/master/review_sentiment_BERT_using_TF(17_June_20).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import The Essentials**

In [1]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random
import matplotlib.pyplot as plt


In [2]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 40kB 2.9MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.4-cp36-none-any.whl size=30114 sha256=67f3293e104e8c9a633df73df993890ef3fa62658e7106f11209e2f04c446e86
  Stored in directory: /root/.cache/pip/wheels/cf/3f/4d/79d7735015a5f523648df90d871ce8e89a7df8185f7703eeab
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=d88a51b478f0c118181e3dd1156024598f626771602a3150d2074d09b3b2a930
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19473 sha256=ff9399d113623b399b3d1566a79a46d244fa0b59d6044bd95a0535b6f464c253
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.1MB 6.8MB/s 


In [5]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert

# **Loading The Dataset**

In [6]:
data=pd.read_csv('Restaurant_Reviews.tsv', delimiter='\t')

In [7]:
data.head(5)

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [9]:
data.shape

(1000, 2)

# **Cleaning Of The Dataset**

In [11]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    # Removing the @
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    # Removing the URL links
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
    # Keeping only letters
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
    # Removing additional whitespaces
    tweet = re.sub(r" +", ' ', tweet)
    return tweet 

In [12]:
data_clean=[clean_tweet(tweet) for tweet in data.Review]

In [13]:
data_clean

['Wow... Loved this place.',
 'Crust is not good.',
 'Not tasty and the texture was just nasty.',
 'Stopped by during the late May bank holiday off Rick Steve recommendation and loved it.',
 'The selection on the menu was great and so were the prices.',
 'Now I am getting angry and I want my damn pho.',
 "Honeslty it didn't taste THAT fresh. ",
 'The potatoes were like rubber and you could tell they had been made up ahead of time being kept under a warmer.',
 'The fries were great too.',
 'A great touch.',
 'Service was very prompt.',
 'Would not go back.',
 'The cashier had no care what so ever on what I had to say it still ended up being wayyy overpriced.',
 'I tried the Cape Cod ravoli chicken with cranberry...mmmm!',
 'I was disgusted because I was pretty sure that was human hair.',
 'I was shocked because no signs indicate cash only.',
 'Highly recommended.',
 'Waitress was a little slow in service.',
 'This place is not worth your time let alone Vegas.',
 'did not like at all.',


In [14]:
data_labels = data.Liked.values #convert list to array 

In [15]:
data_labels[:10]

array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1])

In [16]:
type(data_labels)

numpy.ndarray

#**Tokenizer**

In [17]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [18]:
def encode_sentence(sent):
    return ["[CLS]"] + tokenizer.tokenize(sent) + ["[SEP]"]

In [19]:
data_inputs = [encode_sentence(sentence) for sentence in data_clean] #list comprehension

In [20]:
data_inputs[:10]

[['[CLS]', 'wow', '.', '.', '.', 'loved', 'this', 'place', '.', '[SEP]'],
 ['[CLS]', 'crust', 'is', 'not', 'good', '.', '[SEP]'],
 ['[CLS]',
  'not',
  'ta',
  '##sty',
  'and',
  'the',
  'texture',
  'was',
  'just',
  'nasty',
  '.',
  '[SEP]'],
 ['[CLS]',
  'stopped',
  'by',
  'during',
  'the',
  'late',
  'may',
  'bank',
  'holiday',
  'off',
  'rick',
  'steve',
  'recommendation',
  'and',
  'loved',
  'it',
  '.',
  '[SEP]'],
 ['[CLS]',
  'the',
  'selection',
  'on',
  'the',
  'menu',
  'was',
  'great',
  'and',
  'so',
  'were',
  'the',
  'prices',
  '.',
  '[SEP]'],
 ['[CLS]',
  'now',
  'i',
  'am',
  'getting',
  'angry',
  'and',
  'i',
  'want',
  'my',
  'damn',
  'ph',
  '##o',
  '.',
  '[SEP]'],
 ['[CLS]',
  'hon',
  '##es',
  '##lty',
  'it',
  'didn',
  "'",
  't',
  'taste',
  'that',
  'fresh',
  '.',
  '[SEP]'],
 ['[CLS]',
  'the',
  'potatoes',
  'were',
  'like',
  'rubber',
  'and',
  'you',
  'could',
  'tell',
  'they',
  'had',
  'been',
  'made',
  '

In [21]:
def get_ids(tokens):
    return tokenizer.convert_tokens_to_ids(tokens)

def get_mask(tokens):
    return np.char.not_equal(tokens, "[PAD]").astype(int)
#np.char.not_equal  used to comparision

def get_segments(tokens):
    seg_ids = []
    current_seg_id = 0
    for tok in tokens:
        seg_ids.append(current_seg_id)
        if tok == "[SEP]":
            current_seg_id = 1-current_seg_id # turns 1 into 0 and vice versa
    return seg_ids

In [22]:
data_with_len = [[sent, data_labels[i], len(sent)]
                 for i, sent in enumerate(data_inputs)]
random.shuffle(data_with_len)
data_with_len.sort(key=lambda x: x[2])
sorted_all = [([get_ids(sent_lab[0]),
                get_mask(sent_lab[0]),
                get_segments(sent_lab[0])],
               sent_lab[1])
              for sent_lab in data_with_len if sent_lab[2] > 7]

In [23]:
data_with_len[-10:]

[[['[CLS]',
   'hot',
   'dishes',
   'are',
   'not',
   'hot',
   'cold',
   'dishes',
   'are',
   'close',
   'to',
   'room',
   'te',
   '##mp',
   '.',
   'i',
   'watched',
   'staff',
   'prepare',
   'food',
   'with',
   'bare',
   'hands',
   'no',
   'gloves',
   '.',
   'everything',
   'is',
   'deep',
   'fried',
   'in',
   'oil',
   '.',
   '[SEP]'],
  0,
  34],
 [['[CLS]',
   'the',
   'descriptions',
   'said',
   'yu',
   '##m',
   'yu',
   '##m',
   'sauce',
   'and',
   'another',
   'said',
   'eel',
   'sauce',
   'yet',
   'another',
   'said',
   'spicy',
   'mayo',
   '.',
   '.',
   '.',
   'well',
   'none',
   'of',
   'the',
   'rolls',
   'had',
   'sauce',
   '##s',
   'on',
   'them',
   '.',
   '[SEP]'],
  0,
  34],
 [['[CLS]',
   'i',
   'personally',
   'love',
   'the',
   'hum',
   '##mus',
   'pit',
   '##a',
   'ba',
   '##kla',
   '##va',
   'fa',
   '##la',
   '##fe',
   '##ls',
   'and',
   'baba',
   'gan',
   '##ous',
   '##h',
   'it',
  

In [24]:
# A list is a type of iterator so it can be used as generator for a dataset
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32))

In [26]:
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE,
                                       padded_shapes=((3, None), ()),
                                       padding_values=(0, 0))

In [27]:
NB_BATCHES = math.ceil(len(sorted_all) / BATCH_SIZE)
NB_BATCHES_TEST = NB_BATCHES // 10
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

In [28]:
next(iter(train_dataset))

(<tf.Tensor: shape=(32, 3, 9), dtype=int32, numpy=
 array([[[  101,  2833,  2001,  2061, 27571,  7716,  2094,  1012,   102],
         [    1,     1,     1,     1,     1,     1,     1,     1,     1],
         [    0,     0,     0,     0,     0,     0,     0,     0,     0]],
 
        [[  101,  1996, 26666,  2003,  2498,  2569, 18971,  1012,   102],
         [    1,     1,     1,     1,     1,     1,     1,     1,     1],
         [    0,     0,     0,     0,     0,     0,     0,     0,     0]],
 
        [[  101,  2079,  2025,  5949,  2115,  2769,  2182,   999,   102],
         [    1,     1,     1,     1,     1,     1,     1,     1,     1],
         [    0,     0,     0,     0,     0,     0,     0,     0,     0]],
 
        [[  101,  2037, 20209, 24182,  2003,  2061,  2204,   999,   102],
         [    1,     1,     1,     1,     1,     1,     1,     1,     1],
         [    0,     0,     0,     0,     0,     0,     0,     0,     0]],
 
        [[  101,  2204,  3643,  2307,  2833,  230

# **Model Building**

In [45]:
class DCNNBERTEmbedding(tf.keras.Model):
    
    def __init__(self,
                 nb_filters=50,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 name="dcnn"):
        super(DCNNBERTEmbedding, self).__init__(name=name)
        
        self.bert_layer = hub.KerasLayer(
            "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
            trainable=False)

        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2,
                                    padding="valid",
                                    activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4,
                                      padding="valid",
                                      activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")
    
    def embed_with_bert(self, all_tokens):
        _, embs = self.bert_layer([all_tokens[:, 0, :],
                                   all_tokens[:, 1, :],
                                   all_tokens[:, 2, :]])
        return embs

    def call(self, inputs, training):
        x = self.embed_with_bert(inputs)

        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)
        
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

In [46]:
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2

DROPOUT_RATE = 0.2

BATCH_SIZE = 32
NB_EPOCHS = 5

In [47]:
Dcnn = DCNNBERTEmbedding(nb_filters=NB_FILTERS,
                         FFN_units=FFN_UNITS,
                         nb_classes=NB_CLASSES,
                         dropout_rate=DROPOUT_RATE)

In [48]:
if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])
else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

In [49]:
checkpoint_path = "BERT/ckpt_bert_embedding/"

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!!")

Latest checkpoint restored!!


In [50]:
class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        ckpt_manager.save()
        print("Checkpoint saved at {}.".format(checkpoint_path))

In [51]:
Dcnn.fit(train_dataset,
         epochs=NB_EPOCHS,
         callbacks=[MyCustomCallback()])

Epoch 1/5
     26/Unknown - 2s 93ms/step - loss: 0.0809 - accuracy: 0.9730Checkpoint saved at BERT/ckpt_bert_embedding/.
26/26 [==============================] - 4s 157ms/step - loss: 0.0809 - accuracy: 0.9730
Epoch 2/5
26/26 [==============================] - 4s 169ms/step - loss: 0.0163 - accuracy: 0.9988
Epoch 3/5
26/26 [==============================] - 5s 182ms/step - loss: 0.0149 - accuracy: 0.9975
Epoch 4/5
26/26 [==============================] - 4s 143ms/step - loss: 0.0135 - accuracy: 0.9963
Epoch 5/5
26/26 [==============================] - 10s 365ms/step - loss: 0.0063 - accuracy: 0.9988


In [42]:
results = Dcnn.evaluate(test_dataset)
print(results)

2/2 [==============================] - 0s 45ms/step - loss: 0.5316 - accuracy: 0.7969
[0.5315546989440918, 0.796875]


# **Result Analysis**

In [52]:
def get_prediction(sentence):
    tokens = encode_sentence(sentence)

    input_ids = get_ids(tokens)
    input_mask = get_mask(tokens)
    segment_ids = get_segments(tokens)

    inputs = tf.stack(
        [tf.cast(input_ids, dtype=tf.int32),
         tf.cast(input_mask, dtype=tf.int32),
         tf.cast(segment_ids, dtype=tf.int32)],
         axis=0)
    inputs = tf.expand_dims(inputs, 0) # simulates a batch

    output = Dcnn(inputs, training=False)

    sentiment = math.floor(output*2)

    if sentiment == 0:
        print("Output of the model: {}\nPredicted sentiment: negative".format(
            output))
    elif sentiment == 1:
        print("Output of the model: {}\nPredicted sentiment: positive".format(
            output))

In [53]:
get_prediction("taniya is an amazing deep learning engineer.")

Output of the model: [[0.99972934]]
Predicted sentiment: positive
